In [1]:
import numpy as np
import pandas as pd
import tensorflow_recommenders as tfrs
import tensorflow as tf
from typing import Dict, Text
import pickle as pk
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
import pathlib
import os

user_file = "\\Data\\Combined Data.csv"
caregiver_file = "\\Data\\Processed Caregiver Data.csv"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_dataframe = pd.read_csv(str(parent_path) + user_file)
caregiver_dataframe = pd.read_csv(str(parent_path) + caregiver_file)



In [3]:
unique_masalah_user = np.unique(' '.join(user_dataframe['Tipe_Masalah']).split(' '))
unique_masalah_caregiver = np.unique(' '.join(user_dataframe['Caregiver_Tipe_Masalah']).split(' '))
np.delete(unique_masalah_caregiver, np.where(unique_masalah_caregiver == ''))
unique_masalah_caregiver = np.delete(unique_masalah_caregiver, 0)

unique_masalah_caregiver = unique_masalah_caregiver.tolist()

for index, col in enumerate(unique_masalah_caregiver):
        unique_masalah_caregiver[index] = 'Caregiver-'+col


In [4]:
unique_masalah_caregiver

['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus',
 'Caregiver-Depresi',
 'Caregiver-Gangguan-kecemasan',
 'Caregiver-Gangguan-makan',
 'Caregiver-Gangguan-stres-pascatrauma',
 'Caregiver-Skizofrenia']

In [5]:
unique_masalah_caregiver = ['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus',
 'Caregiver-Depresi',
 'Caregiver-Gangguan-kecemasan',
 'Caregiver-Gangguan-makan',
 'Caregiver-Gangguan-stres-pascatrauma',
 'Caregiver-Skizofrenia']

In [6]:
def convert_categorical_data(df, col='Tipe_Masalah'):
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(col=='Tipe_Masalah'):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [7]:
convert_categorical_data(user_dataframe)
convert_categorical_data(user_dataframe, col='Caregiver_Tipe_Masalah')
convert_categorical_data(caregiver_dataframe, col='Caregiver_Tipe_Masalah')


user_dataframe.drop(["USER_ID"], 1,inplace=True)
user_dataframe.drop(["CAREGIVER_ID"], 1,inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11056\1340759235.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_dataframe.drop(["USER_ID"], 1,inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11056\1340759235.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_dataframe.drop(["CAREGIVER_ID"], 1,inplace=True)


In [8]:

user_dataframe.head()



,Gender,Age,Caregiver_Gender,Caregiver_Age,ADHD-Hiperaktif-dan-kurang-fokus,Depresi,Gangguan-kecemasan,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,Pria,33,Pria,19,1,0,0,0,0,0,1,0,0,0,0,0
1,Pria,33,Pria,35,1,0,0,0,0,0,1,0,1,0,1,0
2,Pria,29,Perempuan,33,0,0,0,1,1,0,0,0,0,1,1,0
3,Pria,29,Pria,38,0,0,0,1,1,0,1,0,0,1,0,0
4,Perempuan,32,Pria,35,1,0,1,0,0,0,1,0,1,0,1,0


In [132]:
user_dataframe.dtypes

Gender                                        object
Age                                            int64
Caregiver_Gender                              object
Caregiver_Age                                  int64
ADHD-Hiperaktif-dan-kurang-fokus               int32
Depresi                                        int32
Gangguan-kecemasan                             int32
Gangguan-makan                                 int32
Gangguan-stres-pascatrauma                     int32
Skizofrenia                                    int32
Caregiver-ADHD-Hiperaktif-dan-kurang-fokus     int32
Caregiver-Depresi                              int32
Caregiver-Gangguan-kecemasan                   int32
Caregiver-Gangguan-makan                       int32
Caregiver-Gangguan-stres-pascatrauma           int32
Caregiver-Skizofrenia                          int32
dtype: object

In [9]:
caregiver_dataframe.head()

,CAREGIVER_ID,Caregiver_Gender,Caregiver_Age,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia
0,1,Perempuan,28,1,0,0,1,1,0
1,2,Pria,33,0,1,0,0,0,1
2,3,Pria,19,1,0,0,0,0,0
3,4,Perempuan,18,0,0,0,1,0,1
4,5,Perempuan,36,0,0,1,0,0,1


In [10]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  if(batch_size == 0 ):
    return ds
  ds = ds.batch(batch_size)
  return ds

In [11]:
unbatched_user_ds = df_to_dataset(user_dataframe, batch_size=0)

In [12]:
batch_size = 5# A small batch sized is used for demonstration purposes
caregiver_ds = df_to_dataset(caregiver_dataframe, batch_size=batch_size,  shuffle=False)
user_ds = df_to_dataset(user_dataframe, batch_size=batch_size)

In [13]:
feature_columns = []

number_feature = ["Age"]
number_feature += unique_masalah_user.tolist()
print(unique_masalah_caregiver)
# numeric cols
for header in number_feature:
    feature_columns.append(feature_column.numeric_column(header))

age_feature = ["Age"]
for col in age_feature:
    age = feature_column.numeric_column(col)
    age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
    feature_columns.append(age_buckets)

['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']


In [14]:
def generate_user_dense_feature():
    feature_columns = []

    number_feature = ["Age"]
    number_feature += unique_masalah_user.tolist()
    # numeric cols
    for header in number_feature:
        feature_columns.append(feature_column.numeric_column(header))

    age_feature = ["Age"]
    for col in age_feature:
        age = feature_column.numeric_column(col)
        age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
        feature_columns.append(age_buckets)

    indicator_column_names = ['Gender']
    for col_name in indicator_column_names:
      categorical_column = feature_column.categorical_column_with_vocabulary_list(
          col_name, user_dataframe[col_name].unique())
      indicator_column = feature_column.indicator_column(categorical_column)
      feature_columns.append(indicator_column)
    
    return tf.keras.layers.DenseFeatures(feature_columns)

In [15]:
def generate_caregiver_dense_feature():
    feature_columns = []

    number_feature = ["Caregiver_Age"]
    number_feature += unique_masalah_caregiver
    # numeric cols

    for header in number_feature:
        feature_columns.append(feature_column.numeric_column(header))

    age_feature = ["Caregiver_Age"]
    for col in age_feature:
        age = feature_column.numeric_column(col)
        age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
        feature_columns.append(age_buckets)

    col_name = 'Caregiver_Gender'
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
          col_name, user_dataframe[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
    
    return  tf.keras.layers.DenseFeatures(feature_columns)

In [16]:
feature_layer_caregiver= generate_caregiver_dense_feature()
feature_layer_user= generate_user_dense_feature()

In [95]:

class RecomendModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    embedding_dimension = 32

    # Set up a model for representing users.
    self.user_model = tf.keras.Sequential([
        self.generate_user_dense_feature(),
        layers.Dense(64),
        layers.Dense(32, activation="linear")
    ])

    # Set up a model for representing caregiver.
    self.caregiver_model = tf.keras.Sequential([
        self.generate_caregiver_dense_feature(),
        layers.Dense(64),
        layers.Dense(32, activation="linear")
    ])

    # Set up a task to optimize the model and compute metrics.
    self.task = tfrs.tasks.Retrieval(
      metrics=tfrs.metrics.FactorizedTopK(
        candidates=caregiver_ds.batch(5).map(self.caregiver_model)
      )
    )
    

  def generate_user_dense_feature(self):
    feature_columns = []

    number_feature = ["Age"]
    number_feature += unique_masalah_user.tolist()
    # numeric cols
    for header in number_feature:
        feature_columns.append(feature_column.numeric_column(header))

    age_feature = ["Age"]
    for col in age_feature:
        age = feature_column.numeric_column(col)
        age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
        feature_columns.append(age_buckets)

    indicator_column_names = ['Gender']
    for col_name in indicator_column_names:
      categorical_column = feature_column.categorical_column_with_vocabulary_list(
          col_name, user_dataframe[col_name].unique())
      indicator_column = feature_column.indicator_column(categorical_column)
      feature_columns.append(indicator_column)
    
    return tf.keras.layers.DenseFeatures(feature_columns)

  def generate_caregiver_dense_feature(self):
    feature_columns = []

    number_feature = ["Caregiver_Age"]
    number_feature += unique_masalah_caregiver
    # numeric cols

    for header in number_feature:
        feature_columns.append(feature_column.numeric_column(header))

    age_feature = ["Caregiver_Age"]
    for col in age_feature:
        age = feature_column.numeric_column(col)
        age_buckets = feature_column.bucketized_column(age, boundaries=[17, 21, 25, 29, 33, 37, 41, 46])
        feature_columns.append(age_buckets)

    col_name = 'Caregiver_Gender'
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
          col_name, user_dataframe[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
    
    return  tf.keras.layers.DenseFeatures(feature_columns)

  def call(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model({
        'Age':[features['Age']],
        'Gender':[features['Gender']],
        'ADHD-Hiperaktif-dan-kurang-fokus':[features['ADHD-Hiperaktif-dan-kurang-fokus']],
        'Depresi':[features['Depresi']],
        'Gangguan-kecemasan':[features['Gangguan-kecemasan']],
        'Gangguan-makan':[features['Gangguan-makan']],
        'Gangguan-stres-pascatrauma':[features['Gangguan-stres-pascatrauma']],
        'Skizofrenia':[features['Skizofrenia']]
    })
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_caregiver_embeddings = self.caregiver_model({
        'Caregiver_Age':[features['Caregiver_Age']],
        'Caregiver_Gender':[features['Caregiver_Gender']],
        'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus':[features['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus']],
        'Caregiver-Depresi':[features['Caregiver-Depresi']],
        'Caregiver-Gangguan-kecemasan':[features['Caregiver-Gangguan-kecemasan']],
        'Caregiver-Gangguan-makan':[features['Caregiver-Gangguan-makan']],
        'Caregiver-Gangguan-stres-pascatrauma':[features['Caregiver-Gangguan-stres-pascatrauma']],
        'Caregiver-Skizofrenia':[features['Caregiver-Skizofrenia']]
        
    })
    return tf.concat([positive_caregiver_embeddings, user_embeddings],0)

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    
    caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
    user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    
    
    # We pick out the user features and pass them into the user model.
    
    user_embeddings = self.user_model({
        'Age':features['Age'],
        'Gender':features['Gender'],
        'ADHD-Hiperaktif-dan-kurang-fokus':features['ADHD-Hiperaktif-dan-kurang-fokus'],
        'Depresi':features['Depresi'],
        'Gangguan-kecemasan':features['Gangguan-kecemasan'],
        'Gangguan-makan':features['Gangguan-makan'],
        'Gangguan-stres-pascatrauma':features['Gangguan-stres-pascatrauma'],
        'Skizofrenia':features['Skizofrenia']
    })
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_caregiver_embeddings = self.caregiver_model({
        'Caregiver_Age':features['Caregiver_Age'],
        'Caregiver_Gender':features['Caregiver_Gender'],
        'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus':features['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'],
        'Caregiver-Depresi':features['Caregiver-Depresi'],
        'Caregiver-Gangguan-kecemasan':features['Caregiver-Gangguan-kecemasan'],
        'Caregiver-Gangguan-makan':features['Caregiver-Gangguan-makan'],
        'Caregiver-Gangguan-stres-pascatrauma':features['Caregiver-Gangguan-stres-pascatrauma'],
        'Caregiver-Skizofrenia':features['Caregiver-Skizofrenia']
        
    })
    

    # The task computes the loss and the metrics.

    return self.task(user_embeddings, positive_caregiver_embeddings, compute_metrics=not training)

In [87]:
caregiver_features = ['Caregiver_Gender', 'Caregiver_Age',  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
user_features = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']

In [88]:

for feature_batch in caregiver_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus'])

Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
A batch of ages: tf.Tensor([1 0 1 0 0], shape=(5,), dtype=int32)


In [89]:
import tensorflow_datasets as tfds

In [90]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [91]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(user_ds, 200, train_split=0.8, val_split=0.2, test_split=0,)

In [110]:
model = RecomendModel()

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

Epoch 1/50
40/40 [==============================] - 1s 10ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 2874.5940 - regularization_loss: 0.0000e+00 - total_loss: 2874.5940
Epoch 2/50
40/40 [==============================] - 0s 10ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 44.6202 - regularization_loss: 0.0000e+00 - total_loss: 44.6202
Epoch 3/50
40/40 [==============================] - 0s 10ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorize

40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 8.1960 - regularization_loss: 0.0000e+00 - total_loss: 8.1960
Epoch 11/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 7.8650 - regularization_loss: 0.0000e+00 - total_loss: 7.8650
Epoch 12/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_catego

40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.5812 - regularization_loss: 0.0000e+00 - total_loss: 6.5812
Epoch 30/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.9164 - regularization_loss: 0.0000e+00 - total_loss: 6.9164
Epoch 31/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_catego

40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.4637 - regularization_loss: 0.0000e+00 - total_loss: 6.4637
Epoch 49/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 6.3220 - regularization_loss: 0.0000e+00 - total_loss: 6.3220
Epoch 50/50
40/40 [==============================] - 0s 9ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_catego

In [24]:
#model.evaluate(unbatched_user_ds.batch(1), return_dict=True)

In [25]:
for feature_batch in caregiver_ds:
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
Every feature: ['CAREGIVER_ID', 'Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-s

In [26]:
brute_force = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
brute_force.index_from_dataset(  
       caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], model.caregiver_model(features)))
)


In [27]:
a =['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test = dict((el,[value[ix]]) for ix,el in enumerate(a))
print(dictionary_Test)

{'Gender': ['Pria'], 'Age': [30], 'ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Depresi': [1], 'Gangguan-kecemasan': [0], 'Gangguan-makan': [0], 'Gangguan-stres-pascatrauma': [1], 'Skizofrenia': [0]}


In [28]:
#user_model = tf.saved_model("temp/user_model_h5.h5")

In [29]:
a =['Caregiver_Gender', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan','Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test_Caregiver = dict((el,([value[ix]])) for ix,el in enumerate(a))
print(dictionary_Test_Caregiver)

{'Caregiver_Gender': ['Pria'], 'Caregiver_Age': [30], 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Caregiver-Depresi': [1], 'Caregiver-Gangguan-kecemasan': [0], 'Caregiver-Gangguan-makan': [0], 'Caregiver-Gangguan-stres-pascatrauma': [1], 'Caregiver-Skizofrenia': [0]}


In [30]:
# Get predictions for user 42.
a, titles = brute_force(dictionary_Test, k=2)
print(f"Top recommendations: {titles[0]}")

Top recommendations: [11 24]


In [31]:
tensor = model.caregiver_model(dictionary_Test_Caregiver)
tensor.numpy()

array([[-0.27607656,  0.7458805 ,  0.26677436,  0.9401697 , -0.73562324,
        -1.3847635 , -0.22383349,  1.0094435 , -0.72571903,  0.36106765,
        -0.13033764, -0.7086837 ,  0.23650563,  0.06816627,  0.91361624,
         0.3600174 , -0.7846898 , -0.15335628,  0.5097748 , -0.29354534,
         0.2285167 ,  0.8161843 , -1.4397314 ,  0.28100234, -0.3305266 ,
         0.3705951 , -0.29639724, -0.149638  , -0.0569072 , -1.4132204 ,
        -0.7104398 , -1.9605787 ]], dtype=float32)

In [32]:
tensor = model.user_model(dictionary_Test)
tensor.numpy()

array([[-6.9482878e-02,  1.2049565e-01,  1.0947990e+00,  5.1290786e-01,
        -6.7551333e-01,  1.8858364e-01, -3.9104661e-01, -1.4123434e+00,
        -8.1696218e-01, -3.3430433e-01, -4.1981332e-02, -8.0621308e-01,
        -6.2047772e-02,  3.5323638e-01,  1.2322776e-03, -2.6854885e-01,
        -1.1595939e+00, -2.5307143e-01,  1.0091046e+00, -9.4284832e-01,
         6.0354197e-01,  2.7488318e-01, -1.1548140e+00,  5.2200210e-01,
        -2.6280224e-01, -3.4273615e-01, -7.0551366e-02,  8.2959324e-01,
        -9.7372420e-02,  2.6178667e-01, -9.1765445e-01, -1.8516840e-01]],
      dtype=float32)

In [33]:
processed_ds = caregiver_ds.batch(5).map(model.caregiver_model)


In [34]:
model.user_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_features_2 (DenseFeat  multiple                 0         
 ures)                                                           
                                                                 
 dense (Dense)               multiple                  1216      
                                                                 
 dense_1 (Dense)             multiple                  2080      
                                                                 
Total params: 3,296
Trainable params: 3,296
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.caregiver_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_features_3 (DenseFeat  multiple                 0         
 ures)                                                           
                                                                 
 dense_2 (Dense)             multiple                  1216      
                                                                 
 dense_3 (Dense)             multiple                  2080      
                                                                 
Total params: 3,296
Trainable params: 3,296
Non-trainable params: 0
_________________________________________________________________


In [36]:
user_model =tf.keras.models.clone_model(
    model.user_model, input_tensors=None, clone_function=None
)
caregiver_model =tf.keras.models.clone_model(
    model.caregiver_model, input_tensors=None, clone_function=None
)

In [37]:
#try save the entire model
model.save_weights('recommender_weight/')

In [38]:
# recreate Saved the model
new_model = RecomendModel()
new_model.load_weights('recommender_weight/')
new_model.user_model(dictionary_Test)

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-6.9482878e-02,  1.2049565e-01,  1.0947990e+00,  5.1290786e-01,
        -6.7551333e-01,  1.8858364e-01, -3.9104661e-01, -1.4123434e+00,
        -8.1696218e-01, -3.3430433e-01, -4.1981332e-02, -8.0621308e-01,
        -6.2047772e-02,  3.5323638e-01,  1.2322776e-03, -2.6854885e-01,
        -1.1595939e+00, -2.5307143e-01,  1.0091046e+00, -9.4284832e-01,
         6.0354197e-01,  2.7488318e-01, -1.1548140e+00,  5.2200210e-01,
        -2.6280224e-01, -3.4273615e-01, -7.0551366e-02,  8.2959324e-01,
        -9.7372420e-02,  2.6178667e-01, -9.1765445e-01, -1.8516840e-01]],
      dtype=float32)>

In [39]:
model.user_model(dictionary_Test)

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-6.9482878e-02,  1.2049565e-01,  1.0947990e+00,  5.1290786e-01,
        -6.7551333e-01,  1.8858364e-01, -3.9104661e-01, -1.4123434e+00,
        -8.1696218e-01, -3.3430433e-01, -4.1981332e-02, -8.0621308e-01,
        -6.2047772e-02,  3.5323638e-01,  1.2322776e-03, -2.6854885e-01,
        -1.1595939e+00, -2.5307143e-01,  1.0091046e+00, -9.4284832e-01,
         6.0354197e-01,  2.7488318e-01, -1.1548140e+00,  5.2200210e-01,
        -2.6280224e-01, -3.4273615e-01, -7.0551366e-02,  8.2959324e-01,
        -9.7372420e-02,  2.6178667e-01, -9.1765445e-01, -1.8516840e-01]],
      dtype=float32)>

In [112]:
model.save_weights('Recommenders_Weights/recommender.hdf5')

In [114]:
model.caregiver_model.summary()

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_features_57 (DenseFea  multiple                 0         
 tures)                                                          
                                                                 
 dense_110 (Dense)           multiple                  1216      
                                                                 
 dense_111 (Dense)           multiple                  2080      
                                                                 
Total params: 3,296
Trainable params: 3,296
Non-trainable params: 0
_________________________________________________________________


In [115]:
new_model = RecomendModel()
new_model(user_dataframe.iloc[:1].to_dict(orient='records')[0])
new_model.load_weights('Recommenders_Weights/recommender.hdf5')

In [116]:
model.user_model(dictionary_Test)

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.52645266, -0.5632986 ,  1.1485107 ,  0.2544759 , -0.04736781,
         0.2640692 , -0.65278715,  0.5571931 , -0.45335707,  0.34283832,
         0.6640647 , -0.01425563,  0.69695026, -0.0617957 , -0.7743329 ,
        -0.611668  ,  0.521925  ,  0.18709213, -0.17164159,  0.11589288,
        -0.6480204 , -0.24467355, -0.21454862,  0.08349912,  0.69035554,
        -0.19064066,  0.97971475,  0.18080468, -0.41349867, -0.17768407,
         0.2672113 , -0.5956174 ]], dtype=float32)>

In [117]:
new_model.user_model(dictionary_Test)

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.52645266, -0.5632986 ,  1.1485107 ,  0.2544759 , -0.04736781,
         0.2640692 , -0.65278715,  0.5571931 , -0.45335707,  0.34283832,
         0.6640647 , -0.01425563,  0.69695026, -0.0617957 , -0.7743329 ,
        -0.611668  ,  0.521925  ,  0.18709213, -0.17164159,  0.11589288,
        -0.6480204 , -0.24467355, -0.21454862,  0.08349912,  0.69035554,
        -0.19064066,  0.97971475,  0.18080468, -0.41349867, -0.17768407,
         0.2672113 , -0.5956174 ]], dtype=float32)>

In [118]:
new_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
new_model.evaluate(user_ds)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_59/dense_features_61/Caregiver-Gangguan-kecemasan/Reshape' defined at (most recent call last):
    File "C:\Users\ASUS\anaconda3\envs\notebook\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ASUS\anaconda3\envs\notebook\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\asus\anaconda3\envs\notebook\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\users\asus\anaconda3\envs\notebook\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\users\asus\anaconda3\envs\notebook\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\users\asus\anaconda3\envs\notebook\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_

In [120]:
y_true = [[0, 1, 2], [0, 0, 0]]
y_pred = [[-18.6, 0.51, -1.2], [2.94, -12.8, 192]]
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(y_true, y_pred).numpy()

33.02085

In [131]:
tf.reshape([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25], [5])






InvalidArgumentError: Input to reshape is a tensor with 25 values, but the requested shape has 5 [Op:Reshape]